In [4]:
# Import libraries
import ee
import json
import numpy as np
import pandas as pd

In [ ]:
# Cloud masking function
def maskL8sr(image):
  cloudShadowBitMask = ee.Number(2).pow(3).int()
  cloudsBitMask = ee.Number(2).pow(5).int()
  qa = image.select('pixel_qa')
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
    qa.bitwiseAnd(cloudsBitMask).eq(0))
  return image.updateMask(mask).select(bands).divide(10000)

In [5]:
with open("Data/geo_data/NUTS_RG_01M_2016_4326_LEVL_2.geojson") as f:
    nuts2_poly = json.load(f)

In [ ]:
nuts2_poly['feature']

In [ ]:
# Load satellite imagery to gdrive
region = ee.Geometry.Polygon(t)
dataset = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(region) \
    .filterDate('2016-01-01', '2016-12-31') \
    .map(maskL8sr)
dataset = dataset.reduce('median')
task = ee.batch.Export.image.toDrive(image=dataset.clip(region),
                                      description="test",
                                      folder="testing",
                                      region=region['coordinates'],
                                      scale=30,
                                      fileFormat='TFRecord',
                                      skipEmptyTiles=True)
task.start()